In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

MAC M1, torch turorial. Originated from https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/Users/alex/miniforge3/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/alex/miniforge3/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <A67B04BC-AB67-333C-B5C9-B63B1EB8B36D> /Users/alex/miniforge3/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/image.so
  Reason: tried: '/Users/alex/miniforge3/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/alex/miniforge3/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/alex/miniforge3/envs/pytorch_m1/lib/python3.8/lib-dynload/../../libjpeg.8.dylib' (no such file), '/Users/alex/miniforge3/envs/pytorch_m1/bin/../lib/libjpeg.8.dylib' (no such file), '/usr/local/lib/libjpeg.8.dylib' (no such file), '/usr/lib/libjpeg.8.dylib' (no such file, not in dyld cache)

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Modelling

In [6]:
# mps for Mac 
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To train a model, we need a loss function and an optimizer.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.



In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308217  [   64/60000]
loss: 2.290139  [ 6464/60000]
loss: 2.276787  [12864/60000]
loss: 2.266814  [19264/60000]
loss: 2.250489  [25664/60000]
loss: 2.227117  [32064/60000]
loss: 2.228038  [38464/60000]
loss: 2.201912  [44864/60000]
loss: 2.207343  [51264/60000]
loss: 2.174462  [57664/60000]
Test Error: 
 Accuracy: 51.2%, Avg loss: 2.165831 

Epoch 2
-------------------------------
loss: 2.175882  [   64/60000]
loss: 2.161796  [ 6464/60000]
loss: 2.112796  [12864/60000]
loss: 2.129132  [19264/60000]
loss: 2.087258  [25664/60000]
loss: 2.023326  [32064/60000]
loss: 2.057154  [38464/60000]
loss: 1.981662  [44864/60000]
loss: 1.988654  [51264/60000]
loss: 1.931417  [57664/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.919548 

Epoch 3
-------------------------------
loss: 1.938552  [   64/60000]
loss: 1.913886  [ 6464/60000]
loss: 1.805294  [12864/60000]
loss: 1.852741  [19264/60000]
loss: 1.750781  [25664/60000]
loss: 1.680241  [32064/600

# Task 1
Build graphic for the learning process. 

# Load/Save models

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
